<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


# **Extract Transform Load (ETL) Lab**


Estimated time needed: **30** minutes


## Objectives

After completing this lab you will be able to:

*   Read CSV and JSON file types.
*   Extract data from the above file types.
*   Transform data.
*   Save the transformed data in a ready-to-load format which data engineers can use to load into an RDBMS.


Import the required modules and functions


In [22]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

## Download Files


In [35]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip

--2022-01-19 09:06:14--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2707 (2.6K) [application/zip]
Saving to: ‘source.zip’

source.zip          100%[===================>]   2.64K  --.-KB/s    in 0s      

2022-01-19 09:06:14 (69.4 MB/s) - ‘source.zip’ saved [2707/2707]



## Unzip Files


In [36]:
!unzip source.zip

Archive:  source.zip
  inflating: source3.json            
  inflating: source1.csv             
  inflating: source2.csv             
  inflating: source3.csv             
  inflating: source1.json            
  inflating: source2.json            
  inflating: source1.xml             
  inflating: source2.xml             
  inflating: source3.xml             


## Set Paths


In [23]:
tmpfile    = "temp.tmp"               # file used to store all extracted data
logfile    = "logfile.txt"            # all event logs will be stored in this file
targetfile = "transformed_data.csv"   # file where transformed data is stored

## Extract


### CSV Extract Function


In [24]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

### JSON Extract Function


In [25]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

### XML Extract Function


In [26]:
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=["name", "height", "weight"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        name = person.find("name").text
        height = float(person.find("height").text)
        weight = float(person.find("weight").text)
        dataframe = dataframe.append({"name":name, "height":height, "weight":weight}, ignore_index=True)
    return dataframe

### Extract Function


In [27]:
def extract():
    extracted_data = pd.DataFrame(columns=['name','height','weight']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data

## Transform


The transform function does the following tasks.

1.  Convert height which is in inches to millimeter
2.  Convert weight which is in pounds to kilograms


In [28]:
def transform(data):
        #Convert height which is in inches to millimeter
        #Convert the datatype of the column into float
        #data.height = data.height.astype(float)
        #Convert inches to meters and round off to two decimals(one inch is 0.0254 meters)
        data['height'] = round(data.height * 0.0254,2)
        
        #Convert weight which is in pounds to kilograms
        #Convert the datatype of the column into float
        #data.weight = data.weight.astype(float)
        #Convert pounds to kilograms and round off to two decimals(one pound is 0.45359237 kilograms)
        data['weight'] = round(data.weight * 0.45359237,2)
        return data

## Loading


In [37]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)  

## Logging


In [30]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

## Running ETL Process


In [38]:
log("ETL Job Started")

In [39]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

,name,height,weight,Unnamed: 0
0,alex,65.78,112.99,NaN
1,ajay,71.52,136.49,NaN
2,alice,69.40,153.03,NaN
3,ravi,68.22,142.34,NaN
4,joe,67.79,144.30,NaN
5,alex,65.78,112.99,NaN
6,ajay,71.52,136.49,NaN
7,alice,69.40,153.03,NaN
8,ravi,68.22,142.34,NaN
9,joe,67.79,144.30,NaN


In [40]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data 

,name,height,weight,Unnamed: 0
0,alex,1.67,51.25,NaN
1,ajay,1.82,61.91,NaN
2,alice,1.76,69.41,NaN
3,ravi,1.73,64.56,NaN
4,joe,1.72,65.45,NaN
5,alex,1.67,51.25,NaN
6,ajay,1.82,61.91,NaN
7,alice,1.76,69.41,NaN
8,ravi,1.73,64.56,NaN
9,joe,1.72,65.45,NaN


In [41]:
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

In [42]:
log("ETL Job Ended")

# Exercise


Using the example above complete the exercise below.


## Download Files


In [71]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

--2022-01-19 09:20:00--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4.1K) [application/zip]
Saving to: ‘datasource.zip.1’

datasource.zip.1    100%[===================>]   4.15K  --.-KB/s    in 0s      

2022-01-19 09:20:00 (120 MB/s) - ‘datasource.zip.1’ saved [4249/4249]



## Unzip Files


In [72]:
!unzip datasource.zip -d dealership_data

Archive:  datasource.zip
  inflating: dealership_data/used_car_prices1.csv  
  inflating: dealership_data/used_car_prices2.csv  
  inflating: dealership_data/used_car_prices3.csv  
  inflating: dealership_data/used_car_prices1.json  
  inflating: dealership_data/used_car_prices2.json  
  inflating: dealership_data/used_car_prices3.json  
  inflating: dealership_data/used_car_prices1.xml  
  inflating: dealership_data/used_car_prices2.xml  
  inflating: dealership_data/used_car_prices3.xml  


## About the Data


The file `dealership_data` contains CSV, JSON, and XML files for used car data which contain features named `car_model`, `year_of_manufacture`, `price`, and `fuel`.


## Set Paths


In [73]:
tmpfile    = "dealership_temp.tmp"               # file used to store all extracted data
logfile    = "dealership_logfile.txt"            # all event logs will be stored in this file
targetfile = "dealership_transformed_data.csv"   # file where transformed data is stored

## Extract


### Question 1: CSV Extract Function


In [82]:
# Add the CSV extract function below
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    print(dataframe)
    return dataframe

<details><summary>Click here for the solution</summary>

```
    
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe
```

</details>


### Question 2: JSON Extract Function


In [75]:
# Add the JSON extract function below
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

<details><summary>Click here for the solution</summary>

```
    
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe
```

</details>


### Question 3: XML Extract Function


In [76]:
# Add the XML extract function below, it is the same as the xml extract function above but the column names need to be renamed.
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=["car_model", "year_of_manufacture", "price", "fuel"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for car in root:
        car_model = car.find("car_model").text
        year_of_manufacture = int(car.find("year_of_manufacture").text)
        price = float(car.find("price").text)
        fuel = car.find("fuel").text
        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel": fuel}, ignore_index=True)
    return dataframe

<details><summary>Click here for the solution</summary>

```
    
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        car_model = person.find("car_model").text
        year_of_manufacture = int(person.find("year_of_manufacture").text)
        price = float(person.find("price").text)
        fuel = person.find("fuel").text
        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, ignore_index=True)
    return dataframe
```

</details>


### Question 4: Extract Function

Call the specific extract functions you created above by replacing the `ADD_FUNCTION_CALL` with the proper function call.


In [84]:
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        print(extracted_data)
    #process all json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
        print(extracted_data)

    #process all xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        print(extracted_data)

    return extracted_data

<details><summary>Click here for the solution</summary>

```
    
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data
```

</details>


## Transform


### Question 5: Transform

Round the `price` columns to 2 decimal places


In [78]:
# Add the transform function below
def transform(data):
    data['price'] = round(data.price, 2)
    return data

<details><summary>Click here for the solution</summary>

```

def transform(data):
        data['price'] = round(data.price, 2)
        return data
```

</details>


## Loading


### Question 6: Load


In [79]:
# Add the load function below
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)

<details><summary>Click here for the solution</summary>

```

def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)  
```

</details>


## Logging


### Question 7: Log

Make sure to change the name of the logfile to the one specified in the set paths section. Change the timestamp order to Hour-Minute-Second-Monthname-Day-Year.


In [80]:
# Add the log function below
def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y' #Hour-Minute-Second-MonthName-Day-Year
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n') 

<details><summary>Click here for the solution</summary>

```

def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y' #Hour-Minute-Second-MonthName-Day-Year
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n') 
```

</details>


## Running ETL Process


### Question 8: ETL Process

Run all functions to extract, transform, and load the data. Make sure to log all events using the `log` function. Place your code under each comment.


In [85]:
# Log that you have started the ETL process
log("ETL Job Started")

# Log that you have started the Extract step
log("Extract phase Started")

# Call the Extract function
extracted_data = extract()

# Log that you have completed the Extract step
log("Extract phase Ended")


# Log that you have started the Transform step
log("Transform phase Started")

# Call the Transform function
transformed_data = transform(extracted_data)

# Log that you have completed the Transform step
log("Transform phase Ended")

# Log that you have started the Load step
log("Load phase Started")

# Call the Load function
load(targetfile,transformed_data)

# Log that you have completed the Load step
log("Load phase Ended")

# Log that you have completed the ETL process
log("ETL Job Ended")

       car_model  year_of_manufacture         price    fuel
0           ritz                 2014   5000.000000  Petrol
1            sx4                 2013   7089.552239  Diesel
2           ciaz                 2017  10820.895522  Petrol
3        wagon r                 2011   4253.731343  Petrol
4          swift                 2014   6865.671642  Diesel
5  vitara brezza                 2018  13805.970149  Diesel
6           ciaz                 2015  10074.626866  Petrol
7        s cross                 2015   9701.492537  Diesel
8           ciaz                 2016  13059.701493  Diesel
9           ciaz                 2015  11119.402985  Diesel
       car_model year_of_manufacture         price    fuel
0           ritz                2014   5000.000000  Petrol
1            sx4                2013   7089.552239  Diesel
2           ciaz                2017  10820.895522  Petrol
3        wagon r                2011   4253.731343  Petrol
4          swift                2014   6865.6

<details><summary>Click here for the solution</summary>

```

log("ETL Job Started")

log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")

log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")

log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

log("ETL Job Ended")
```

</details>


## Authors


Ramesh Sannareddy

Joseph Santarcangelo

Azim Hirjani


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                 |
| ----------------- | ------- | ----------------- | ---------------------------------- |
| 2020-11-25        | 0.1     | Ramesh Sannareddy | Created initial version of the lab |


Copyright © 2020 IBM Corporation. This notebook and its source code are released under the terms of the [MIT License](https://cognitiveclass.ai/mit-license?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork23455645-2021-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork-23455645&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
